In [1]:
import heapq
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd

In [ ]:
class Model():
    '''
    This is the central model
    '''
    def __init__(self, A, M, Pij, ArrLst, RhoMtx, Beta, B_, Gamma, Mu, N, Delta, D_, WARMTIME, RUNTIME):     
        #print('init')
        self.A, self.M, self.Pij, self.ArrLst, self.RhoMtx, self.Beta, self.B_, self.Gamma, self.Mu, self.N, self.Delta, self.D_ = \
            A, M, Pij, ArrLst, RhoMtx, Beta, B_, Gamma, Mu, N, Delta, D_
        self.TIMELIMIT = WARMTIME + RUNTIME
        self.EPISODES = EPISODES
        self.NUMBERPERF = NUMBERPERF
        #print(self.TIMELIMIT)
        self.reset()
        #self.epi = 0
        #self.Performance = [0] * EPISODES
        
        
    def reset(self):
        A, M, ArrLst = self.A, self.M, self.ArrLst
        self.T = 0 # time cursor
        self.formerT = 0
        self.areas = list(range(self.A))
        # serve performance parameters
        #self.normalBikes= M
        # maintainance performance parameters
        #self.idleRate, self.BP, self.RC, self.DP = 0, 0, 0, 0
        
        # indicators of time stamp
        [self.nt, self.bt, self.it, self.bpt, self.rct, self.dpt] = [0]*6
        self.enormalBikes = 0
        # maintainance performance parameters
        self.eidleRate, self.eBP, self.eRC, self.eDP = 0, 0, 0, 0
        self.earrivals, self.elostCustomers = [0]*A, [0]*A
        
        self.state1 = [int(M/A)]*A + [0]*3 # Nis, BP, FC, and DP
        self.state2 = [[0]*A for i in range(A)] # Rijs
        self.F = [] # time to be empty for fixing queue
        heapq.heapify(self.F)
        
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(A):
            heapq.heappush(self.scheduler, [random.expovariate(ArrLst[i]), -1, i, i])
        #self.stateRecord = self.state1[:A] + self.state2[0] + self.state2[1] + self.state1[-3:]
        #return self.state1, self.state2, self.T
        
    def setRecord(self, kind):
        if kind == 1:
            self.enormalBikes += self.normalBikes * (self.T - self.nt) 
            self.normalBikes, self.nt = self.normalBikes - 1, self.T
            self.eBP += self.BP * (self.T - self.bpt) 
            self.BP, self.bpt = self.BP + 1, self.T
        elif kind == 2:
            if self.RC == 0: 
                self.eidleRate += (self.T - self.it) 
            self.eBP += self.BP * (self.T - self.bpt) 
            self.BP, self.bpt = self.BP - self.B_, self.T
            self.eRC += self.RC * (self.T - self.rct) 
            self.RC, self.rct = self.RC + self.B_, self.T
        elif kind == 3:
            self.eRC += self.RC * (self.T - self.rct) 
            self.RC, self.rct = self.RC - 1, self.T
            self.eDP += self.DP * (self.T - self.dpt) 
            self.DP, self.dpt = self.DP + 1, self.T
            if self.RC == 0: 
                self.it = self.T
        elif kind == 4: 
            self.enormalBikes += self.normalBikes * (self.T - self.nt) 
            self.normalBikes, self.nt = self.normalBikes + self.D_, self.T
            self.eDP += self.DP * (self.T - self.dpt) 
            self.DP, self.dpt = self.DP - self.D_, self.T
        elif kind == -10:
            self.arrivals += 1
        elif kind == -11:
            self.lostCustomers += 1
        else: print('fucking wrong')
    
    def storeData(self):
        with open(FileAdd+'/systemPerformance' +NAME+str(self.para)+'.csv', 'w', newline='') as fout:
            writer = csv.writer(fout)
            for p in self.Performance:
                writer.writerow(p)
        
    def returnPerformance(self, Performance, parameter):
        re = []
        perf = np.array(Performance)
        for i in range(self.NUMBERPERF):
            re.append([parameter, np.average(perf[:, i]), np.var(perf[:,i])**0.5])
        return re
        
    def simulate(self, parameter):
        #print('simulate', self.Beta)
        Performance = self.EPISODES * [0]
        #self.para = para
        if WriteFile:
            with open(FileAdd, 'w', newline='') as fout:
                writer = csv.writer(fout)
                for i in range(self.EPISODES):
                    self.reset()
                    self.epi = i
                    #print('simulating')
                    while self.T <= self.TIMELIMIT:
                        # print(self.T)
                        self.stepForward()
                        #self.getRecord()
                        writer.writerow(self.getRecord())  
        else:
            plt.ion()
            for i in range(self.EPISODES):
                self.epi = i
                self.reset()
                while self.T <= self.TIMELIMIT:
                    self.stepForward()
                    plt.scatter(self.T, self.)
                    #self.getRecord()
                #if self.Beta == 0: self.enormalBikes = self.M * self.TIMELIMIT
                lst = [i/self.TIMELIMIT for i in [self.enormalBikes, (self.M*self.TIMELIMIT - self.enormalBikes), self.TIMELIMIT*self.lostCustomers/self.arrivals, self.eidleRate, self.eBP, self.eRC, self.eDP]]
                Performance[i] = lst
            #print(self.TIMELIMIT)

        #self.storeData()
        #print(self.Performance)
        return self.returnPerformance(Performance, parameter)
    
    def addEvent(self, kind):
        if kind == -1:
            next_time = random.expovariate(self.ArrLst[self.start]) + self.T
            start, end = self.start, self.start
        elif kind == 1:
            next_time = random.expovariate(self.RhoMtx[self.start][self.terminal]) + self.T
            start, end = self.start, self.terminal
        elif kind == 2: 
            next_time = random.expovariate(self.Gamma)
            next_time += self.T
            start, end = 'b', 'f'
        elif kind == 3:
            next_time = random.expovariate(self.Mu) 
            if self.state1[-2] < self.N:
                next_time += self.T 
                heapq.heappush(self.F, next_time)
            else: 
                next_time += heapq.heappop(self.F)
                heapq.heappush(self.F, next_time)
            start, end = 'f', 'd'
        else: 
            next_time = random.expovariate(self.Delta)
            next_time += self.T
            start, end = 'd', 'ni'
        heapq.heappush(self.scheduler, [next_time, kind, start, end])
        
    def bikeArr(self):
        self.state2[self.start][self.terminal] -= 1
        if random.random()<self.Beta:
            self.state1[-3] += 1
            heapq.heappop(self.scheduler)
            if self.state1[-3] == self.B_:
                self.addEvent(2)
            self.setRecord(1)
        else:
            self.state1[self.terminal] += 1
            heapq.heappop(self.scheduler)
    def BPover(self):
        heapq.heappop(self.scheduler)
        for i in range(self.B_): 
            self.state1[-3] -= 1
            self.addEvent(3) 
            self.state1[-2] += 1
        if self.state1[-3] >= self.B_: self.addEvent(2)
        self.setRecord(2)
    def repair(self):
        heapq.heappop(self.scheduler)
        if self.state1[-2] <= self.N: heapq.heappop(self.F)
        self.state1[-2] -= 1
        self.state1[-1] += 1
        if self.state1[-1] == self.D_:
            self.addEvent(4)
        self.setRecord(3)
    def DPover(self):
        heapq.heappop(self.scheduler)
        self.state1[-1] -= self.D_
        for i in range(self.A): self.state1[i] += int(self.D_/self.A)
        if self.state1[-1] >= self.D_: self.addEvent(4)
        self.setRecord(4)
    def cusArr(self):
        self.setRecord(-10)
        if self.state1[self.start] == 0:  # 但没车
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
            self.setRecord(-11)
        else:
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
            # below use self.terminal to represent the target
            self.terminal = random.choices(self.areas, weights=self.Pij[self.start], k=1)[0]
            self.state1[self.start] -= 1
            self.state2[self.start][self.terminal] += 1 
            self.addEvent(1)

    def stepForward(self):
        event = self.scheduler[0]
        #print(event)
        self.T, self.kind, self.start, self.terminal = event[0], event[1], event[2], event[3]
        '''
        kind of events:
        -1: customer ride a bike away
         1: a bike arrives at any area
         2: BP greater than B_
         3: a bike is fixed
         4: DP greater than D_
        '''
        if self.kind == 1: 
            self.bikeArr() # 顾客骑行到达
        elif self.kind == 2:
            self.BPover() # 坏车达到阈值
        elif self.kind == 3:
            self.repair() # 修好一辆车
        elif self.kind == 4:
            self.DPover() # 再分配
        else:# 顾客到达
            self.cusArr() #顾客到达

        return self.state1, self.state2, self.T

In [ ]:
class Model():
    '''
    This is the distributed model
    '''
    def __init__(self, A, M, Pij, ArrLst, RhoMtx, Beta, Mu, N, Theta):     
        self.timeLimit = WARMTIME + RUNTIME
        self.areas = list(range(A))
        self.epi = 0
        
        self.Performance = [0] * EPISODES
        self.A, self.M, self.Pij, self.ArrLst, self.RhoMtx, self.Beta, self.Mu, self.N, self.Theta = \
            A, M, Pij, ArrLst, RhoMtx, Beta, Mu, N, Theta
        
    def reset(self):
        self.T = 0 # time cursor
        self.formerT = 0
        self.servedCustomers = 0 # number of customer served during runtime in an episode

        self.state1 = [int(M/A)]*A + [0] * A  + [0] # Nis, Bis
        self.state2 = [[0]*A for i in range(A)] # Rijs
        self.stateRecord = self.state1[:-1] + self.state2[0] + self.state2[1] + [self.state1[-1]]
        self.F = [] # time to be empty for fixing queue
        heapq.heapify(self.F)
        
        self.normalBikes, self.brokenBikes, self.idleRate, self.arrivals, self.lostCustomers = [self.M] + [0] * 4
        [self.nt, self.bt, self.it] = [0]*3
        self.enormalBikes, self.ebrokenBikes, self.eidleRate = [0] * 3
        
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(A):
            heapq.heappush(self.scheduler, [random.expovariate(ArrLst[i]), -1, i, i])
        heapq.heappush(self.scheduler, [random.expovariate(Theta), 2, 0, 1])
        return self.state1, self.state2, self.T
    
    def setRecord(self, kind):
        if kind == 1:
            self.enormalBikes += self.normalBikes * (self.T - self.nt) 
            self.normalBikes, self.nt = self.normalBikes - 1, self.T
        elif kind == 20:
            self.it = self.T
        elif kind == 21:
            self.eidleRate += self.T - self.it
            self.it = self.T
        elif kind == 3:
            self.enormalBikes += self.normalBikes * (self.T - self.nt) 
            self.normalBikes, self.nt = self.normalBikes + 1, self.T
        elif kind == -10:
            self.arrivals += 1
        elif kind == -11:
            self.lostCustomers += 1
        else: print('fucking wrong')
    
    def storeData(self):
        with open(FileAdd+'/systemPerformance' +NAME+str(self.para)+'.csv', 'w') as fout:
            writer = csv.writer(fout)
            for p in self.Performance:
                writer.writerow(p)
        
    def returnPerformance(self):
        re = []
        perf = np.array(self.Performance)
        for i in range(NUMBERPERF):
            re.append([self.para, np.average(perf[:, i]), np.var(perf[:,i])**0.5])
        return re
        
    def simulate(self, para):
        #print('simulate')
        self.para = para
        if WriteFile:
            with open(FileAdd, 'w', newline='') as fout:
                writer = csv.writer(fout)
                for i in range(EPISODES):
                    self.reset()
                    self.epi = i
                    print('simulating')
                    while self.T <= self.timeLimit:
                        # print(self.T)
                        self.stepForward()
                        # self.getRecord()
                        writer.writerow(self.getRecord())  
        else:
            for i in range(EPISODES):
                self.epi = i
                self.reset()
                while self.T <= self.timeLimit:
                    self.stepForward()
                self.Performance[self.epi] = [self.enormalBikes/self.timeLimit, self.M - self.enormalBikes/self.timeLimit, self.lostCustomers/self.arrivals, self.eidleRate/self.timeLimit]
        #self.storeData()
        #print(self.returnPerformance())
        return self.returnPerformance()
    
    def pop2(self):
        self.s = []
        for e in self.scheduler:
            if e[1] == 2: 
                self.setRecord(21)
                continue
            else: self.s.append(e)
        heapq.heapify(self.s)
        self.scheduler = self.s

    def addEvent(self, kind):
        if kind == -1:
            next_time = random.expovariate(self.ArrLst[self.start]) + self.T
            start, end = self.start, self.start
        elif kind == 1:
            next_time = random.expovariate(self.RhoMtx[self.start][self.terminal]) + self.T
            start, end = self.start, self.terminal
        elif kind == 2: 
            next_time = random.expovariate(self.Theta)
            next_time += self.T
            start, end = self.terminal, (self.terminal+1)%self.A
            #print('add event 2')
        else:
            next_time = random.expovariate(self.Mu) 
            if self.state1[self.A+self.terminal] < self.N:
                next_time += self.T 
                heapq.heappush(self.F, next_time)
            else: 
                next_time += heapq.heappop(self.F)
                heapq.heappush(self.F, next_time)
            start, end = self.terminal, self.terminal
        heapq.heappush(self.scheduler, [next_time, kind, start, end])
        
    def bikeArr(self):
        self.state2[self.start][self.terminal] -= 1
        heapq.heappop(self.scheduler)
        if random.random()<self.Beta:
            #if self.state1[-1] == self.terminal and self.F != []: 
            if self.state1[-1] == self.terminal: 
                # delete next truck move event
                self.pop2()
                self.addEvent(3)
            self.state1[self.A+self.terminal] += 1
            self.setRecord(1)
        else:
            self.state1[self.terminal] += 1
    def truckArr(self):
        heapq.heappop(self.scheduler)
        self.state1[-1] = self.terminal
        broken = self.state1[self.A+self.terminal]
        if broken > 0: # 给坏车们逐一确定维修好的时间
            self.state1[self.A+self.terminal] = 0 
            for i in range(broken):
                self.addEvent(3)
                self.state1[self.A+self.terminal] += 1
            self.setRecord(21)
        else:
            self.addEvent(2)
    def repair(self):
        heapq.heappop(self.scheduler)
        if self.state1[self.A+self.terminal] <= self.N: heapq.heappop(self.F)
        self.state1[self.A+self.terminal] -= 1
        self.state1[self.terminal] += 1
        if self.state1[self.A+self.terminal] == 0:
            self.addEvent(2)
            self.setRecord(20)
        self.setRecord(3)
    def cusArr(self):
        #print(self.state1, self.state2)
        #print('------------------------')
        self.setRecord(-10)
        if self.state1[self.start] == 0:  # 但没车
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
            self.setRecord(-11)
        else:
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
            # below use self.terminal to represent the target
            self.terminal = random.choices(self.areas, weights=self.Pij[self.start], k=1)[0]
            self.state1[self.start] -= 1
            self.state2[self.start][self.terminal] += 1 
            self.addEvent(1)

    def stepForward(self):
        event = self.scheduler[0]
        #print(event)
        self.T, self.kind, self.start, self.terminal = event[0], event[1], event[2], event[3]
        '''
        kind of events:
        -1: customer ride a bike away
         1: a bike arrives at any area
         2: truck arrives
         3: a bike is fixed
        '''
        if self.kind == 1: 
            self.bikeArr() # 顾客骑行到达
        elif self.kind == 2:
            self.truckArr() # 维修车到达
        elif self.kind == 3:
            self.repair() # 修好一辆车
        else:# 顾客到达
            self.cusArr() #顾客到达